## Scraping MedReg

Das BAG stellt zwar zum MedReg Rohdatensätze zur Verfügung, doch fehlen diesen Angaben über
den Arbeitsort der verzeichneten Ärzte. Eine Anfrage für anonymisierte Rohdaten 
des MedRegs wurde vom BAG negativ beantwortet. 
Deshalb werde ich die Daten des MedRegs komplett scrapen.

Da im MedReg neben den Ärzten auch Tierärzte, Zahnärzte und andere Medizinalpersonen
verzeichnet sind, muss die Datenbank in einem zweiten Schritt bereinigt werden. 

Eine Abfrage nur nach Ärzten, bzw. Psychiatern (Facharzttitel) erscheint mir nicht 
möglich, da die Abfrage nach der ID-Nummer der Einträge erfolgt. Ich konnte nicht erkennen,
das bestimmte Bereiche für bestimmte Arztkategorien reserviert wären. 

Ein Problem: wie finde ich die höchste verzeichnete Nummer im Register. 
Eine manuelles Eingrenzen mit der Abfrage verschiedener Nummern ergibt, dass die PID wohl fortlaufend vergeben wird. 
Mit der PID 110'000 ist ein Arzt verzeichnet, dessen deutsches Diplom im Juni 2019 anerkannt worden war. Die letzen erfassten Einträge liegen (Ende Oktober 2019) im Bereich von 113'100. Sicherheitshalber werden deshalb einige Seiten mehr abgefragt. 

Hier ein erster Anlauf zum Bau eines Scraping-Programms. 

Zuerst laden wir die notwendigen Python-Module. 


In [1]:
import requests
import time
import random

Erster Test. Was kommt da von der Seite. 

In [2]:
r = requests.get("https://www.medregom.admin.ch/DE/Detail/Detail?pid=1234")
r.text

'<!DOCTYPE html PUBLIC \'-//W3C//DTD XHTML 1.0 Transitional//EN\' \'http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd\'>\n\t\t\t\t\t<html xmlns=\'http://www.w3.org/1999/xhtml\' lang=\'de\' xml:lang=\'de\'>\n\t\t\t\t\t\t<head>\n\t\t\t\t\t\t\t<title>Bundesamt f&uuml;r Informatik und Telekommunikation BIT</title>\n\t\t\t\t\t\t\t<meta http-equiv=\'Content-Type\' content=\'text/html; charset=utf-8\' />\n\t\t\t\t\t\t\t<meta name=\'author\' content=\'Bundesamt f&uuml;r Informatik und Telekommunikation BIT\' />\n\t\t\t\t\t\t\t<meta name=\'GENERATOR\' content=\'BIT-FEA\' />\n\t\t\t\t\t\t\t<meta name=\'content-language\' content=\'de\' />\n\t\t\t\t\t\t\t<meta name=\'audience\' content=\'all\' />\n\t\t\t\t\t\t\t<meta name=\'revisit-after\' content=\'7 days\' />\n\t\t\t\t\t\t\t<meta name=\'robots\' content=\' index,follow \' />\n\t\t\t\t\t\t\t<meta name=\'description\' content=\'Master Prototype f&uuml;r statische Bit-Web Sites\' />\n\t\t\t\t\t\t\t<meta name=\'abstract\' content=\'Master Pro

*Sehr geehrter Benutzer, sehr geehrte Benutzerin</p>\n\t\t\t\t\t\t\t\t\t<p>Die Bundesbeh&ouml;rden setzen f&uuml;r den  Schutz der Internetauftritte Systeme ein, welche die Zugriffe auf Ihre  Korrektheit hin &uuml;berpr&uuml;fen. Der von Ihnen durchgef&uuml;hrte Zugriff wurde als  unkorrekt eingestuft und daher blockiert. Falls Sie der Ansicht sind, dass Sie Zugriff auf die blockierte Seite  haben sollten, so wenden Sie sich bitte unter der Angaben der Nummer <b>C-7127910736182721775</b> per E-Mail an das <a href=\'mailto:servicedesk@bit.admin.ch?subject=SCOD-B-000000002 - WAF Incident Nr. C-7127910736182721775\'>Service Desk </a>des Bundesamt f&uuml;r Informatik und  Telekommunikation. Wir  werden diesen Vorfall gerne &uuml;berpr&uuml;fen und Sie anschliessend innerhalb von  einer Woche benachrichtigen.*
   
   
Aha, Zugriff blockiert. Kein Datensatz, so wie wenn man diesselbe Abfrage im Browser macht. Ich muss wohl die Abfrage anders als geplant ausführen. 

Trotzdem noch einige Checks:

In [30]:
r.status_code

200

In [3]:
r.headers

{'Date': 'Sun, 16 Feb 2020 13:27:26 GMT', 'X-Powered-By': 'PHP/5.2.14', 'Content-Length': '2915', 'Keep-Alive': 'timeout=15, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'text/html', 'Set-Cookie': 'TS01a2e852=014346c5073aacc533ff012c76d5fcf59ce42d7a6b1a1f47ae043303896d0b811c0638a0b8179779e741de95feea352ab82df73531; Path=/, TS01a2e852_28=016ec04b89d988ffb0806b7b495efcf36632465c5d3e94b76b91f790e2e9adc6c8edf3799121352284fd46b7371ec784714b92ecf4; Path=/', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'gzip'}

In [32]:
r.encoding

'ISO-8859-1'

Wir die Requests-Abfrage als solche erkannt? Ich vermute, ich muss der Webseite einen Browser vorgaukeln. Klappt das evtl. wenn ich einen Header mitsende?


In [2]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", 
    "Accept-Language": "de-CH,en;q=0.8,fr-CH;q=0.5", 
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:70.0) Gecko/20100101 Firefox/70.0"
}

r = requests.get("https://www.medregom.admin.ch/DE/Detail/Detail?pid=1244", headers = headers)
r.content

b'<?xml version="1.0" encoding="utf-8"?>\r\n<div>\r\n  <div style="min-height:51px">\r\n    <h3>Hartmann, Elfriede\xc2\xa0<img class="geschlecht-icon" src="../../Images/venus-solid.png" /></h3>\r\n    <div class="favorit detail">\xc2\xa0</div>\r\n    <div style="clear:left" />\r\n  </div>\r\n  <p>Nationalit\xc3\xa4t: Schweiz (CH)<br />Sprachkenntnisse: Deutsch<br />GLN: 7601000351302<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (fr\xc3\xbcher EAN) ">\xc2\xa0</a><br />UID: <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gr\xc3\xbcnden identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge f\xc3\xbcr die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen gef\x

**Nachtrag 5.11.2019: Die Daten sind jetzt da. Entweder habe ich sie beim ersten Durchgang nicht gesehen oder sie waren nicht da.**

*Keine Blockade mehr, aber trotzdem nicht die richtigen Daten. 
Eigentlich müsste das der Datensatz https://www.medregom.admin.ch/DE/Detail/Detail?pid=1244 lauten auf:
*Hartmann, Elfriede
Nationalität: Schweiz (CH)
Sprachkenntnisse: Deutsch
GLN: 7601000351302*

*Doch leider kommen diese Daten nirgends vor. Ist das vielleicht nicht im Klartext gespeichert?*

In [47]:
r = requests.get("https://www.medregom.admin.ch/DE/Detail/Detail?pid=1244", headers = headers, stream = True)

In [48]:
r.raw

In [44]:
r.raw.read(100)

b'tent="no-cache"/>\r\n<meta http-equiv="Expires" content="-1"/>\r\n<meta http-equiv="CacheControl" conten'

In [46]:
with open("medreg_testraw", 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)

Sieht auch nicht besser aus, ich finde im gespeicherten File keine Daten. 
Eigentlich müsste der Datensatz so aussehen (Code aus dem Browser kopiert):

<?xml version="1.0" encoding="utf-8"?>
<div>
  <div style="min-height:51px">
    <h3>Hartmann, Elfriede <img class="geschlecht-icon" src="../../Images/venus-solid.png" /></h3>
    <div class="favorit detail"> </div>
    <div style="clear:left" />
  </div>
  <p>Nationalität: Schweiz (CH)<br />Sprachkenntnisse: Deutsch<br />GLN: 7601000351302<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (früher EAN) "> </a><br />UID: <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gründen identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge für die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen geführt."> </a></p>
</div>
<div class="right">
  <div id="noresult_overlay">
    <div id="noresult_message" />
  </div>
  <div id="map" />
</div>
<div class="left">
  <ul class="detail-navigation">
    <li id="5">Zahnärztin/Zahnarzt</li>
  </ul>
  <div id="detail_5" class="ac-detail-data">
    <div>
      <table class="detail-table">
        <thead>
          <tr>
            <td class="head">Beruf</td>
            <td class="head">Erteilungsjahr</td>
            <td class="head">Erteilungsland</td>
            <td class="head">Typ</td>
          </tr>
        </thead>
        <tbody>
          <tr>
            <td>Zahnärztin/Zahnarzt</td>
            <td>1987</td>
            <td>Schweiz</td>
            <td>Eidgenössisches Diplom</td>
          </tr>
          <tr>
            <th class="in-table-head">Weiterbildungstitel</th>
            <th />
            <th />
            <th />
          </tr>
          <tr>
            <td colspan="4">Keine Angaben vorhanden</td>
          </tr>
          <tr>
            <th class="in-table-head">Weitere Qualifikationen (privatrechtliche Weiterbildung)</th>
            <th />
            <th />
            <th />
          </tr>
          <tr>
            <td colspan="4">Keine Angaben vorhanden</td>
          </tr>
        </tbody>
      </table>
    </div>
    <hr />
    <h4>Status der Berufsausübungsbewilligung<a class="info" title="Erteilt, aktiv = gültige Berufsausübungsbewilligung für diesen Kanton, aktiv / Erteilt, inaktiv = gültige Berufsausübungsbewilligung für diesen Kanton, inaktiv / Keine Bewilligung, Bewilligung entzogen = Berufsausübungsbewilligung in diesem Kanton entzogen / Keine Bewilligung, Bewilligung verweigert = Berufsausübungsbewilligung in diesem Kanton verweigert / Keine Bewilligung = keine Bewilligung beantragt oder Berufsausübungsverbot"> </a></h4>
    <h4>Erteilt</h4>
    <ol style="list-style-type: circle">
      <li>Zürich
                (1988), , MedBG, privatwirtschaftliche Berufsausübung in eigener fachlicher Verantwortung</li>
    </ol>
    <h4>Direktabgabe von Arzneimitteln gemäss kant. Bestimmungen (Selbstdispensation)<a class="info" title="Keine Selbstdispensation: Darf nur in Sonderfällen (z.B. Notfällen) Medikamente abgeben / Keine Angaben vorhanden: keine kantonalen Daten vorhanden /  Bewilligung erteilt für Adresse(n): Darf Medikamente abgeben, d.h. ist zum Führen einer Privatapotheke berechtigt. / Es gelten die kantonalen Bestimmungen."> </a></h4>keine Selbstdispensation<h4>Bezug von Betäubungsmitteln<a class="info" title="Dieses Feld dient zur Information der Pharmagrossisten für die Belieferung "> </a></h4>Berechtigung erteilt für Kanton(e): Zürich<hr /><h4>Adresse(n)</h4><ol class="address"><li><span class="outdent">Bewilligungskanton: Zürich</span></li><li title="Edisonstrasse 12,8050 Zürich,ZH" class="address-item"><span class="outdent">A.
            </span><div>Elfriede Hartmann<br />Zahnarztpraxis<br />Edisonstrasse 12<br />8050 Zürich<br />Telefon: 044 312 24 56<br />Fax: 044 312 24 56<br />UID: <a class="info" title="UID der juristischen Person/Betrieb welche dieser Adresse entspricht."> </a></div></li></ol></div>
</div>
<div class="clearRight"></div>


<APM_DO_NOT_TOUCH>

<script language="javascript">

(function(){

window.NSk=!!window.NSk;try{(function(){try{var ll,Ll,Ol=1;for(var Sl=0;Sl<Ll;++Sl)Ol+=3;ll=Ol;window.ss===ll&&(window.ss=++ll)}catch(il){window.ss=ll}var jl=!0;function Jl(l){!l||document.visibilityState&&"visible"!==document.visibilityState||(jl=!1,document.cookie="brav=ad");return jl}function sL(){}Jl(window[sL.name]===sL);Jl("function"!==typeof ie9rgb4);Jl(/\x3c/.test(function(){return"\x3c"})&!/x3d/.test(function(){return"'x3'+'d';"}));
var SL=window.attachEvent||/mobi/i.test(window["\x6e\x61vi\x67a\x74\x6f\x72"]["\x75\x73e\x72A\x67\x65\x6et"]),IL=+new Date+6E5,jL,JL,lo,Io=setTimeout,lO=SL?3E4:6E3;function LO(){if(!document.querySelector)return!0;var l=+new Date,z=l>IL;if(z)return Jl(!1);z=JL&&!lo&&jL+lO<l;z=Jl(z);jL=l;JL||(JL=!0,Io(function(){JL=!1},1));return z}LO();
document.addEventListener&&document.addEventListener("visibilitychange",function(l){document.visibilityState&&("hidden"===document.visibilityState&&l.isTrusted?lo=!0:"visible"===document.visibilityState&&(jL=+new Date,lo=!1,LO()))});var oO=[17795081,27611931586,1558153217];function OO(l){l="string"===typeof l?l:l.toString(36);var z=window[l];if(!z.toString)return;var s=""+z;window[l]=function(l,s){JL=!1;return z(l,s)};window[l].toString=function(){return s}}for(var ZO=0;ZO<oO.length;++ZO)OO(oO[ZO]);
Jl(!1!==window.NSk);
(function sO(){if(!LO())return;var z=!1;function s(z){for(var s=0;z--;)s+=_(document.documentElement,null);return s}function _(z,s){var S="vi";s=s||new I;return sl(z,function(z){z.setAttribute("data-"+S,s.ij());return _(z,s)},null)}function I(){this.J_=1;this.i_=0;this.zZ=this.J_;this.c=null;this.ij=function(){this.c=this.i_+this.zZ;if(!isFinite(this.c))return this.reset(),this.ij();this.i_=this.zZ;this.zZ=this.c;this.c=null;return this.zZ};this.reset=function(){this.J_++;this.i_=0;this.zZ=this.J_}}
var J=!1;function S(z,s){if(!LO())return;var S=document.createElement(z);s=s||document.body;s.appendChild(S);S&&S.style&&(S.style.display="none");LO()}function zl(s,S){if(!LO())return;S=S||s;var _="|";function I(z){z=z.split(_);var s=[];for(var S=0;S<z.length;++S){var J="",ol=z[S].split(",");for(var zl=0;zl<ol.length;++zl)J+=ol[zl][zl];s.push(J)}return s}var zl=0,sl="datalist,details,embed,figure,hrimg,strong,article,formaddress|audio,blockquote,area,source,input|canvas,form,link,tbase,option,details,article";
sl.split(_);sl=I(sl);sl=new RegExp(sl.join(_),"g");while(sl.exec(s))sl=new RegExp((""+new Date)[8],"g"),z&&(J=LO()),++zl;return LO()?S(zl&&1):void 0}function sl(z,s,_){if(!LO())return;(_=_||J)&&S("div",z);z=z.children;var I=0;for(var zl in z){_=z[zl];try{_ instanceof HTMLElement&&(s(_),++I)}catch(sl){}}return LO()?I:void 0}zl(sO,s);LO()})();var SO=18;window.L_={zi:"08f2d0f59781b000276d829d7c53f8e46336ed5bcde1661000b7aada14d98de2a2bca79a5e510c46e1ef5e0f02071bb46101cda07886652c7d0077756b52435e4df7961a0604840dcaa1a6799255306b42a31a0737cdb3685f7c6a5ac3d64a0929d5f36d4fc7166c5fe9f13fcff45388cd3c9c182074327c34b278b5166dd1d3c37063338599d9c1e46d3eb077ff15711c288ef3e0113fe4f0575f6d98a44152b93788298807821db24e35ac4997b262f5bca4c4e4aa9351"};function L(l){return 773>l}
function O(l){var z=arguments.length,s=[];for(var _=1;_<z;++_)s.push(arguments[_]-l);return String.fromCharCode.apply(String,s)}function Z(l,z){l+=z;return l.toString(36)}(function iO(z){return z?0:iO(z)*iO(z)})(LO());})();}catch(x){document.cookie='brav=oex'+x;}finally{ie9rgb4=void(0);};function ie9rgb4(a,b){return a>>b>>0};

})();

</script>
</APM_DO_NOT_TOUCH>
<script type="text/javascript" src="/TSbd/08fe5c3c79ab2000db8b6c67fa48a68ece1dbd913a28f58f568b05629d735651c8128d3e7589157f?type=3"></script>
<script type="text/javascript">
	$(document).ready(function() {
		try {
			var _json_test = JSON;
		} catch (e) {
			document.createStyleSheet('/Content/ie8_compatview.css');
		}
	});
</script>



Ich passe den User-Header nochmals an, ganz reduziert. 

In [14]:
headers = {"User-Agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:70.0) Gecko/20100101 Firefox/70.0"
          }

r = requests.get("https://www.medregom.admin.ch/DE/Detail/Detail?pid=1244", headers = headers)
r.content

b'<?xml version="1.0" encoding="utf-8"?>\r\n<div>\r\n  <div style="min-height:51px">\r\n    <h3>Hartmann, Elfriede\xc2\xa0<img class="geschlecht-icon" src="../../Images/venus-solid.png" /></h3>\r\n    <div class="favorit detail">\xc2\xa0</div>\r\n    <div style="clear:left" />\r\n  </div>\r\n  <p>Nationalit\xc3\xa4t: Schweiz (CH)<br />Sprachkenntnisse: Deutsch<br />GLN: 7601000351302<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (fr\xc3\xbcher EAN) ">\xc2\xa0</a><br />UID: <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gr\xc3\xbcnden identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge f\xc3\xbcr die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen gef\x

Mit *r.text* kommt die Meldung: 
> Please enable JavaScript to view the page content. 

Mit *r.content* klappt's, wir haben das File vor uns. Hier mal reinkopiert: 

b'<?xml version="1.0" encoding="utf-8"?>\r\n<div>\r\n  <div style="min-height:51px">\r\n    <h3>Hartmann, Elfriede\xc2\xa0<img class="geschlecht-icon" src="../../Images/venus-solid.png" /></h3>\r\n    <div class="favorit detail">\xc2\xa0</div>\r\n    <div style="clear:left" />\r\n  </div>\r\n  <p>Nationalit\xc3\xa4t: Schweiz (CH)<br />Sprachkenntnisse: Deutsch<br />GLN: 7601000351302<a class="info" title="Global Location Number: eindeutige Nummer einer Medizinalperson (fr\xc3\xbcher EAN) ">\xc2\xa0</a><br />UID: <a class="info" title="UID = Unternehmens-Identifikationsnummer. Die UID dient der eindeutigen Identifikation von Unternehmen und wird jeder organisatorischen oder institutionellen Einheit, die aus rechtlichen, administrativen oder statistischen Gr\xc3\xbcnden identifiziert werden muss (UID-Einheit), zugewiesen. Die UID ist die Nachfolge f\xc3\xbcr die MWST-Nr. und die Handelsregisternummer und wird auch von weiteren Verwaltungsregistern, wie z.B. den AHV-Ausgleichskassen gef\xc3\xbchrt.">\xc2\xa0</a></p>\r\n</div>\r\n<div class="right">\r\n  <div id="noresult_overlay">\r\n    <div id="noresult_message" />\r\n  </div>\r\n  <div id="map" />\r\n</div>\r\n<div class="left">\r\n  <ul class="detail-navigation">\r\n    <li id="5">Zahn\xc3\xa4rztin/Zahnarzt</li>\r\n  </ul>\r\n  <div id="detail_5" class="ac-detail-data">\r\n    <div>\r\n      <table class="detail-table">\r\n        <thead>\r\n          <tr>\r\n            <td class="head">Beruf</td>\r\n            <td class="head">Erteilungsjahr</td>\r\n            <td class="head">Erteilungsland</td>\r\n            <td class="head">Typ</td>\r\n          </tr>\r\n        </thead>\r\n        <tbody>\r\n          <tr>\r\n            <td>Zahn\xc3\xa4rztin/Zahnarzt</td>\r\n            <td>1987</td>\r\n            <td>Schweiz</td>\r\n            <td>Eidgen\xc3\xb6ssisches Diplom</td>\r\n          </tr>\r\n          <tr>\r\n            <th class="in-table-head">Weiterbildungstitel</th>\r\n            <th />\r\n            <th />\r\n            <th />\r\n          </tr>\r\n          <tr>\r\n            <td colspan="4">Keine Angaben vorhanden</td>\r\n          </tr>\r\n          <tr>\r\n            <th class="in-table-head">Weitere Qualifikationen (privatrechtliche Weiterbildung)</th>\r\n            <th />\r\n            <th />\r\n            <th />\r\n          </tr>\r\n          <tr>\r\n            <td colspan="4">Keine Angaben vorhanden</td>\r\n          </tr>\r\n        </tbody>\r\n      </table>\r\n    </div>\r\n    <hr />\r\n    <h4>Status der Berufsaus\xc3\xbcbungsbewilligung<a class="info" title="Erteilt, aktiv = g\xc3\xbcltige Berufsaus\xc3\xbcbungsbewilligung f\xc3\xbcr diesen Kanton, aktiv / Erteilt, inaktiv = g\xc3\xbcltige Berufsaus\xc3\xbcbungsbewilligung f\xc3\xbcr diesen Kanton, inaktiv / Keine Bewilligung, Bewilligung entzogen = Berufsaus\xc3\xbcbungsbewilligung in diesem Kanton entzogen / Keine Bewilligung, Bewilligung verweigert = Berufsaus\xc3\xbcbungsbewilligung in diesem Kanton verweigert / Keine Bewilligung = keine Bewilligung beantragt oder Berufsaus\xc3\xbcbungsverbot">\xc2\xa0</a></h4>\r\n    <h4>Erteilt</h4>\r\n    <ol style="list-style-type: circle">\r\n      <li>Z\xc3\xbcrich\r\n                (1988), , MedBG, privatwirtschaftliche Berufsaus\xc3\xbcbung in eigener fachlicher Verantwortung</li>\r\n    </ol>\r\n    <h4>Direktabgabe von Arzneimitteln gem\xc3\xa4ss kant. Bestimmungen (Selbstdispensation)<a class="info" title="Keine Selbstdispensation: Darf nur in Sonderf\xc3\xa4llen (z.B. Notf\xc3\xa4llen) Medikamente abgeben / Keine Angaben vorhanden: keine kantonalen Daten vorhanden /  Bewilligung erteilt f\xc3\xbcr Adresse(n): Darf Medikamente abgeben, d.h. ist zum F\xc3\xbchren einer Privatapotheke berechtigt. / Es gelten die kantonalen Bestimmungen.">\xc2\xa0</a></h4>keine Selbstdispensation<h4>Bezug von Bet\xc3\xa4ubungsmitteln<a class="info" title="Dieses Feld dient zur Information der Pharmagrossisten f\xc3\xbcr die Belieferung ">\xc2\xa0</a></h4>Berechtigung erteilt f\xc3\xbcr Kanton(e): Z\xc3\xbcrich<hr /><h4>Adresse(n)</h4><ol class="address"><li><span class="outdent">Bewilligungskanton: Z\xc3\xbcrich</span></li><li title="Edisonstrasse 12,8050 Z\xc3\xbcrich,ZH" class="address-item"><span class="outdent">A.\r\n            </span><div>Elfriede Hartmann<br />Zahnarztpraxis<br />Edisonstrasse 12<br />8050 Z\xc3\xbcrich<br />Telefon: 044 312 24 56<br />Fax: 044 312 24 56<br />UID: <a class="info" title="UID der juristischen Person/Betrieb welche dieser Adresse entspricht.">\xc2\xa0</a></div></li></ol></div>\r\n</div>\r\n<div class="clearRight"></div>\r\n\r\n<script type="text/javascript">\r\n\t$(document).ready(function() {\r\n\t\ttry {\r\n\t\t\tvar _json_test = JSON;\r\n\t\t} catch (e) {\r\n\t\t\tdocument.createStyleSheet(\'/Content/ie8_compatview.css\');\r\n\t\t}\r\n\t});\r\n</script>\r\n\r\n'

Ok, das scheint zu klappen. Jetzt noch abspeichern und Formatproblem lösen:

In [56]:
r.encoding = 'utf-8'

with open("medreg_testcontent", 'wb', ) as file:
    datensatz = r.content
    file.write(datensatz)

Geschafft: Das File sieht gut aus, auch die Umlaute werden korrekt wiedergeben! (Aufwand von ganz oben bis hier ca. 3,5 Stunden).

Im nächsten Schritt widmen wir uns dem Bau des Scrapers. 

In [7]:
import requests
import random
import csv
from bs4 import BeautifulSoup
from time import sleep

url_stamm = "https://www.medregom.admin.ch/DE/Detail/Detail?pid="
output_path = "/Users/master/Desktop/CAS_DJ/P2_MedReg/MedReg/scraped_data/"
pers_id = 0
protokoll = []
time_delay = random.uniform(1,3)

#Loop für das Erstellen und Abfragen der URL:
for url_id in range(20,22):
    r = requests.get(url_stamm + str(url_id))
    if r.status_code == 200:
        with open(output_path + "medreg_" + str(url_id) + ".html", 'w', encoding = 'utf-8') as f:
            f.write(r.text)
            f.close()
        protokoll.append((url_stamm + str(url_id), r.status_code))
        
    else:
        protokoll.append((url_stamm + str(url_id), r.status_code))
        
        continue
    
    time.sleep(time_delay)

with open(output_path + "scrape_protokoll.csv", 'w') as p:
    writer = csv.writer(p)
    writer.writerows(protokoll)


Funktioniert mit den zwei Files. Aber wegen fehlendem Header wird nur die Meldung runtergeladen, dass Anfrage blockiert wurde. Also nochmals mit Header:

In [15]:
import requests
import random
import csv
from time import sleep

url_stamm = "https://www.medregom.admin.ch/DE/Detail/Detail?pid="
output_path = "/Users/master/Desktop/CAS_DJ/P2_MedReg/MedReg/scraped_data/"
pers_id = 0
protokoll = []
time_delay = random.uniform(2,4)
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}

#Loop für das Erstellen und Abfragen der URL:
for url_id in range(25,28):
    r = requests.get(url_stamm + str(url_id), headers = headers, timeout=10)
    sleep(time_delay)
    if r.status_code == 200:
        with open(output_path + "medreg_" + str(url_id) + ".html", 'w', encoding = 'utf-8') as f:
            f.write(r.text)
            f.close()
        protokoll.append((url_stamm + str(url_id), r.status_code))
        
    else:
        protokoll.append((url_stamm + str(url_id), r.status_code))
        
        continue
    

with open(output_path + "scrape_protokoll.csv", 'w') as p:
    writer = csv.writer(p)
    writer.writerows(protokoll)

Das Scraping funktioniert jetzt nicht richtig. Es kommen Fehlermeldungen 500 zurück für jede Seite, schön vermerkt im Protokoll. Und hier noch im Klartext:

In [16]:
r = requests.get(url_stamm + str(url_id), headers = headers, timeout=10)
r.text

'\r\n\r\n\r\n<!DOCTYPE HTML PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="de">\r\n<head>\r\n\t<meta http-equiv="Content-Type" content="text/xml; charset=UTF-8" />\r\n\t<title>Fehler Erreur errore</title>\r\n\t<link href="/Content/site.css" rel="stylesheet" type="text/css" />\r\n\t<!--[if lte IE 7]>\r\n\t<link href="/Content/ie.css" rel="stylesheet" type="text/css" />\r\n\t<![endif]-->\r\n</head>\r\n<body>\r\n\t<div id="page">\r\n\t\t<div id="pageHeader">\r\n\t\t\t<div class="left"></div>\r\n\t\t\t<div class="right">\r\n\t\t\t\t<div class="appTitle">Bundesverwaltung admin.ch</div>\r\n\t\t\t\t<div class="appTitleEdi">Eidgenössisches Departement des Innern EDI</div>\r\n\t\t\t\t<div class="appTitleBag">Bundesamt für Gesundheit BAG</div>\r\n\t\t\t</div>\r\n\t\t</div>\r\n\t\t<div id="clearHeader"></div>\r\n\t\t<div id="pageNavigation">\r\n\t\t\t<div class="navigation">\r\n\t\t\t\t<ul>\r\n\t\t

Ist der Link ok?

In [18]:
print(url_stamm + str(url_id))

https://www.medregom.admin.ch/DE/Detail/Detail?pid=27


Er scheint ok, aber gibt auch beim manuellen Aufruf die Fehlermeldung im Browser.
Dieser hat funktioniert, beim Testen: https://www.medregom.admin.ch/DE/Detail/Detail?pid=1244
Nach rumdoktern und entdecke ich: die Einträge beginnen erst bei 203. 


In [ ]:
import requests
import random
import csv
from time import sleep

url_stamm = "https://www.medregom.admin.ch/DE/Detail/Detail?pid="
output_path = "/Users/master/Desktop/CAS_DJ/P2_MedReg/MedReg/scraped_data/"
pers_id = 0
protokoll = []
time_delay = random.uniform(1,2)
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}

#Loop für das Erstellen und Abfragen der URL:
for url_id in range(300,310):
    r = requests.get(url_stamm + str(url_id), headers = headers, timeout=10)
    sleep(time_delay)
    if r.status_code == 200:
        with open(output_path + "medreg_" + str(url_id) + ".html", 'w', encoding = 'utf-8') as f:
            f.write(r.text)
            f.close()
        protokoll.append((url_stamm + str(url_id), r.status_code))
        
    else:
        protokoll.append((url_stamm + str(url_id), r.status_code))
        
        continue
    

with open(output_path + "scrape_protokoll.csv", 'w') as p:
    writer = csv.writer(p)
    writer.writerows(protokoll)

Nach Abruf der pid 1196 kommt folgender Fehler:
ReadTimeout: HTTPSConnectionPool(host='www.medregom.admin.ch', port=443): Read timed out. (read timeout=10)

Die Protokolldatei wird nicht erstellt. 



Noch ein Anlauf zu scrapen. Die Änderungen:

1.) Ich setzte den Timeintervall höher, vielleicht wurde der Ladevorgang deshalb abgebrochen.

2.) Und statt if/else für korrekte Reponsecodes (200) passe ich den Code an, dass alle Seiten, auch die Fehlermeldungen gespeichtert werden. 

3.) Ein weitere Anpassung: das csv-Protokoll soll mitlaufend geführt werden, um den Verlust bei Abbruch/Runterfahren des Kernels zu vermeiden. 

Einträge kommen erst ab 203. Deshalb starte ich den neuen Scraper ab 200 und lasse ihn bis Eintrag 2000 laufen. 


In [ ]:
import requests
import random
import csv
from time import sleep

url_stamm = "https://www.medregom.admin.ch/DE/Detail/Detail?pid="
output_path = "/Users/master/Desktop/CAS_DJ/P2_MedReg/MedReg/scraped_data/"
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}

#Loop für das Erstellen und Abfragen der URL:
for url_id in range(960,2000):
    
    url_req = url_stamm + str(url_id) #erstellt die Abfrage-URL für die Datensätze im range(x,y)
    r = requests.get(url_req, headers = headers, timeout=10)
    with open(output_path + "medreg_" + str(url_id) + ".html", 'w', encoding = 'utf-8') as f:
        f.write(r.text)
        f.close()
        
        #protokoll = (url_req, r.status_code)
        #oder
        protokoll = [[url_req, r.status_code]] 
        #es braucht Doppelklammern! Bis ich das rausgefunden habe: 1 Stunde.
    
    with open(output_path + "scrape_protokoll.csv", 'a') as p:
        writer = csv.writer(p, dialect = 'excel')
        writer.writerows(protokoll)
                                      
    sleep(random.uniform(2, 5))
                                     

Nach 959 Einträgen die Meldung:

ConnectionError: HTTPSConnectionPool(host='www.medregom.admin.ch', port=443): Max retries exceeded with url: /DE/Detail/Detail?pid=960 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x1131b5050>: Failed to establish a new connection: [Errno 50] Network is down'))

Vermutlich wurde meine IP wegen zuvieler Anfragen ausgesperrt. Anpassungen im Scraper. Mit Retry sollen Fehler nochmals verarbeitet werden.

In [ ]:
import requests
import random
import csv
from time import sleep

url_stamm = "https://www.medregom.admin.ch/DE/Detail/Detail?pid="
output_path = "/Users/master/Desktop/CAS_DJ/P2_MedReg/MedReg/scraped_data/"

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}

#Loop für das Erstellen und Abfragen der URL:

for url_id in range(1138,10000):
    url_req = url_stamm + str(url_id) #erstellt die Abfrage-URL für die Datensätze im range(x,y)
    try:
        r = requests.get(url_req, headers = headers, timeout=10)
        with open(output_path + "medreg_" + str(url_id) + ".html", 'w', encoding = 'utf-8') as f:
            f.write(r.text)
            f.close()
      
        protokoll = [[url_req, r.status_code]] 
ConnectionError:
        r.status_code =
        with open(output_path + "scrape_protokoll.csv", 'a') as p:
            writer = csv.writer(p, dialect = 'excel')
            writer.writerows(protokoll)
    except requests.exceptions. "Connection refused"
                                      
    sleep(random.uniform(3, 8))
                                     

Nach 333 Items auch hier wieder ausgesperrt:

ReadTimeout: HTTPSConnectionPool(host='www.medregom.admin.ch', port=443): Read timed out. (read timeout=10)

Die Sleep-Zeit erhöhe ich auf 5 bis 8 Sekunden, was die Abfrage von über 100'000 Einträgen natürlich massiv verlängert. Eine zusätzliche Pausenschleife soll die Abfragedauer erhöhen. https://stackoverflow.com/questions/23013220/max-retries-exceeded-with-url-in-requests

In [ ]:
import requests
import random
import csv
import time
from time import sleep


url_stamm = "https://www.medregom.admin.ch/DE/Detail/Detail?pid="
output_path = "/Users/master/Downloads/scraped_data/"
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}

#Loop für das Erstellen und Abfragen der URL:
for url_id in range(50001,150000):
    url_req = url_stamm + str(url_id) #erstellt die Abfrage-URL für die Datensätze im range(x,y)
    try:
        r = requests.get(url_req, headers = headers, timeout=10)
        with open(output_path + "medreg_" + str(url_id) + ".html", 'w', encoding = 'utf-8') as f:
            f.write(r.text)
            f.close()
      
        protokoll = [[url_req, r.status_code]] 

        with open(output_path + "scrape_protokoll.csv", 'a') as p:
            writer = csv.writer(p, dialect = 'excel')
            writer.writerows(protokoll)
            
        #sleep(random.uniform(0, 2))
        #nach 5100 Files, die in ca 14 Stunden runtergeladen wurden, jetzt der Versuch die Sache etwas zu beschleunigen.
    except: 
        r = requests.exceptions.ConnectionError
        print('Connection refused')
        time.sleep(180)
        
        continue


Erkenntnis: offenbar geht es hier nach Zugriffszeit, nicht nach Anzahl Abfragen oder Intervall der Abfragen. 

## Erstes Scraping erfolgreich
Zuerst habe mit einem Zufalls-Intervall von 5-10 Sekunden pro Anfrage gearbeitet. Dabei wurde ich regelmässig nach einigen hundert Downloads blockiert. Mit der Except-Schlaufe und einem time.sleep von 3 Minuten schaffte ich es immerhin, dass das Programm selbstständig weiterlief. 
Doch für rund 3000 Datensätze lief das Programm rund 14 Stunden. Die Abfrage von 150'000 Seiten hätte demnach 29 Tage gedauert!

Erst der Versuch, die einzelnen Abfragen im Eiltempo, ohne Zufalls-Intervall durchzuführen, brachte eine befriedende Geschwindigkeit. Von den PID's 10001 bis 20'0000 wurden die Daten in gut 1:36h runtergeladen. Dabei wurden nur zwei Blockaden verzeichnet. 

Die restlichen Daten bis zum Datensatz 150'000 geht dann in weiteren Schritten relativ schnell. Von 50'000 bis 150'000 wage ich dann in einem Schritt. 

Das gesamte Scraping dauert vom 7.11.2019 00:49 bis 10.11.2019 17:53. Am Schluss liegen im Ordner 149'767 Files (705 MB). Der Scrape startete bei ID 200. Insgesamt fehlen damit 32 Datensätze (oder leere Seiten). Ich werde später entscheiden, ob ich die gezielt nachladen werde. 
